In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from bposd.hgp import hgp
from ldpc.code_util import codewords, compute_code_distance, get_code_parameters, search_cycles
from ldpc.mod2 import rank, reduced_row_echelon, row_basis
from scipy.sparse import csc_matrix
from bposd.css import css_code

import numpy as np
from ldpc import bposd_decoder
import time
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing import Pool

In [2]:
def f(p,a):
    return 1-(1-p)**a

def df(p,a):
    return a*(1-p)**(a-1)

def err(p, N):
    return np.sqrt(p*(1-p)/N)

In [ ]:
import Gibbs_sampler_beta

start = time.time() 

min_n=int(4)
max_n=int(4)

res = 15
beta_min = 1.2
beta_max = 2.3
beta_list = np.logspace(np.log10(beta_min), np.log10(beta_max), res)
iters = (np.zeros(res)+200).astype(np.int64)
t=30

min_n2=2
max_n2=5

THREADS = 7    # Number of parallel threads
LERs = np.zeros([max_n2-min_n2+1,res])
pool = Pool(processes=THREADS)

for n in range(min_n,max_n+1):
    
    b=3 #b-ary tree
    #n=2 # n-level tree
    n_c = 1+b*((b-1)**(n-1)-1)/(b-2)
    parity_check = np.zeros((int(b*((b-1)**(n-1)-1)/(b-2)), int(1+b*((b-1)**(n-1)-1)/(b-2))))
    for k in range(0, int(b*((b-1)**(n-1)-1)/(b-2))):
        parity_check[k][k+1]=1
    parity_check[0][0]=1
    count=0
    for k1 in range(0,int((b*(b-1)**(n-2)-2)/(b-2))):
        for k2 in range(0,int(b-1)):
            count += 1
            parity_check[int(count)][k1]=1

    for n2 in range(min_n2,max_n2+1):
        parity_check2 = np.zeros((int(b*((b-1)**(n2-1)-1)/(b-2)), int(1+b*((b-1)**(n2-1)-1)/(b-2))))
        for k in range(0, int(b*((b-1)**(n2-1)-1)/(b-2))):
            parity_check2[k][k+1]=1
        parity_check2[0][0]=1
        count=0
        for k1 in range(0,int((b*(b-1)**(n2-2)-2)/(b-2))):
            for k2 in range(0,int(b-1)):
                count += 1
                parity_check2[int(count)][k1]=1
        
        code=hgp(h1=parity_check,h2=parity_check2,compute_distance=True)
        
        
        print('Code: {} / {}  , Res: {} / {} '.format(n-min_n+1,max_n-min_n+1,n2-min_n2+1,max_n2-min_n2+1), end='\r')
        for r in range(res):
            params=(code.hx,code.N,code.lx,t,iters[r],beta_list[r],code.N-rank(code.hx))
            args = [params for j in range(THREADS)]
            if __name__ ==  '__main__':
                logical_error_rate = pool.starmap(Gibbs_sampler_beta.Gibbs_sampler_beta, args)
                LERs[n2-min_n2,r] = np.sum(logical_error_rate) / THREADS
pool.close()

LERs_err = np.zeros([max_n2-min_n2+1,res])
for i in range(len(LERs)):
    for k in range(res):
        LERs_err[i,k] =  err(LERs[i,k], iters[k]*THREADS)


end = time.time()
print("It took", end-start, "seconds!")

